In [25]:
# 4.1 FewShotPromptTemplate (13:52)

from langchain.chat_models import ChatOpenAI

# from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import (
    FewShotPromptTemplate,
    FewShotChatMessagePromptTemplate,
)
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

"""
'''
# 1번 방법 (더 편함)
t = PromptTemplate.from_template("What is the capital of {country}")

# 2번 방법
t = PromptTemplate(
    template="What is the capital of {country}",
    input_variables=["country"],
)
'''

t.format(country="France") 
"""

examples = [
    {
        # "question": "What do you know about France?",
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        # "question": "What do you know about Italy?",
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        # "question": "What do you know about Greece?",
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

'''
example_template="""
    Human: {question}
    AI: {answer}
"""

# example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
# example_prompt = PromptTemplate.from_template(example_template)

'''


#4.2 FewShotChatMessagePromptTemplate (07:24)
example_template = [
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}"),
]
example_prompt = ChatPromptTemplate.from_messages(example_template)


# prompt = FewShotPromptTemplate(
#     example_prompt=example_prompt,
#     examples=examples,
#     suffix="Human: What do you know about {country}?",
#     input_variables=["country"],
# )

# prompt.format(country="Germany")


example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give me short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat

chain.invoke({"country": "Korea"})


        I know this:
        There are two Koreas - North Korea and South Korea.
        Capital of South Korea: Seoul
        Capital of North Korea: Pyongyang
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korea - South Korean Won, North Korea - North Korean Won
        

AIMessageChunk(content='\n        I know this:\n        There are two Koreas - North Korea and South Korea.\n        Capital of South Korea: Seoul\n        Capital of North Korea: Pyongyang\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korea - South Korean Won, North Korea - North Korean Won\n        ')